In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

/Users/hoyeon/Github/tickai/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_ds_feat = pd.read_csv('../../data/processed_train_feat.csv', index_col=0)
train_ds_targ = pd.read_csv('../../data/processed_train_targ.csv', index_col=0)

test_ds_feat = pd.read_csv('../../data/processed_test_feat.csv', index_col=0)
test_ds_targ = pd.read_csv('../../data/processed_test_targ.csv', index_col=0)

In [3]:
num_feat = len(train_ds_feat.columns)
num_feat

39

In [24]:
train_high = train_ds_targ.iloc[:, 0].to_numpy().reshape(len(train_ds_feat), 1)
train_low = train_ds_targ.iloc[:, 1].to_numpy().reshape(len(train_ds_feat), 1)

In [37]:
train_ds_feat.values

array([[0.26853269, 0.05380133, 0.26870873, ..., 0.00078402, 0.04153535,
        0.03961086],
       [0.26006913, 0.05276105, 0.2602472 , ..., 0.00075294, 0.03462366,
        0.03924884],
       [0.26756983, 0.04863707, 0.26944033, ..., 0.00078041, 0.0323415 ,
        0.03365286],
       ...,
       [0.27065099, 0.01313258, 0.24017635, ..., 0.00066569, 0.01323654,
        0.00719042],
       [0.23326304, 0.01387759, 0.23084846, ..., 0.00063179, 0.01480145,
        0.00823413],
       [0.24042674, 0.01240516, 0.23613331, ..., 0.00063801, 0.01512748,
        0.00615056]])

In [6]:
class MyDataset(Dataset):
    def __init__(self, feat, targ):
        self.feat = feat.values
        self.targ = targ
        
    def __len__(self):
        return len(self.feat)
    
    def __getitem__(self, idx):
        feat = torch.FloatTensor(self.feat[idx])
        targ = torch.FloatTensor(self.targ[idx])
        return feat, targ

In [7]:
train_ds = MyDataset(train_ds_feat, train_ds_targ)
test_ds = MyDataset(test_ds_feat, test_ds_targ)

train_dl = DataLoader(train_ds, batch_size=20, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=1, shuffle=True)

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [25]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, input_size * 2),
            nn.ReLU(),
            nn.Linear(input_size * 2, input_size * 2),
            nn.ReLU(),
            nn.Linear(input_size * 2, output_size)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(num_feat, 1).to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=39, out_features=78, bias=True)
    (1): ReLU()
    (2): Linear(in_features=78, out_features=78, bias=True)
    (3): ReLU()
    (4): Linear(in_features=78, out_features=1, bias=True)
  )
)


In [15]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [33]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [29]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [34]:
epochs = 20

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dl, model, loss_fn, optimizer)
    test(test_dl, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


KeyError: 1426

In [ ]:
test(test_dl, model, loss_fn)